In [26]:
import pandas as pd
data = pd.read_csv('hsbc_twitter_DATA_not GENERAL.csv', error_bad_lines=False);
data_text = data[['tweet']]
data_text['index'] = data_text.index
documents = data_text

<ipython-input-26-f28b4a75d8b9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['index'] = data_text.index


In [27]:
print(len(documents))
print(documents[:5])

4467
                                               tweet  index
0  @LondOgrad Hello Eugene, we do apologise for t...      0
1  @ncohen4 @CooperativeBank @HSBC @HSBCUKBusines...      1
2  @sharaola Good morning, sorry for your experie...      2
3        @Bounce_BackLoan Still not a peep from HSBC      3
4  @djm2481 @mgcmitchell @HSBC_UK Thanks for the ...      4


In [28]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [29]:
stemmer = SnowballStemmer("english")

In [30]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [31]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['@Debasmita076', 'Hello', 'Debasmita,', 'thank', 'you', 'for', 'writing', 'into', 'us.', 'Address', 'change', 'options', 'are.', '1==', 'if', 'you', 'have', 'registered', 'for', 'Internet', 'Banking', '&amp;', 'have', 'a', 'security', 'device.', 'Please', 'log', 'in', 'â†’Click', 'on', 'Help', '&amp;', 'Support', 'â†’', 'to', 'the', 'left', 'side', 'you', 'have', 'an', 'option', 'Servicesâ†’', 'Send', 'a', 'SECURE', 'MESSAGE.', '(1/3)']


 tokenized and lemmatized document: 
['debasmita', 'hello', 'debasmita', 'thank', 'write', 'address', 'chang', 'option', 'regist', 'internet', 'bank', 'secur', 'devic', 'click', 'help', 'support', 'leav', 'option', 'servicesâ', 'send', 'secur', 'messag']


In [33]:
processed_docs = documents['tweet'].map(preprocess)
processed_docs[:10]

0    [londograd, hello, eugen, apologis, delay, cur...
1    [ncohen, cooperativebank, hsbc, hsbcukbusi, si...
2    [sharaola, good, morn, sorri, experi, send, co...
3                        [bounce_backloan, peep, hsbc]
4    [mgcmitchel, hsbc_uk, thank, mention, paul, ha...
5    [bounce_backloan, evid, get, hsbc, payout, thi...
6    [good, morn, nani, happi, answer, hsbc, bank, ...
7    [jsr_mzp, hello, thank, write, accept, imp, cr...
8    [isrivat, hello, thank, write, request, kind, ...
9    [miss_simmon, cooperativebank, hsbc, donâ, bot...
Name: tweet, dtype: object

In [34]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 apologis
1 check
2 current
3 custom
4 delay
5 eugen
6 hello
7 huge
8 line
9 londograd
10 manag


In [35]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [36]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(4, 1),
 (22, 1),
 (26, 1),
 (28, 1),
 (41, 1),
 (48, 1),
 (49, 1),
 (96, 1),
 (164, 1),
 (230, 1),
 (260, 1),
 (296, 1),
 (320, 1),
 (323, 1),
 (325, 1),
 (331, 2),
 (432, 2)]

In [37]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 4 ("hello") appears 1 time.
Word 22 ("help") appears 1 time.
Word 26 ("send") appears 1 time.
Word 28 ("thank") appears 1 time.
Word 41 ("bank") appears 1 time.
Word 48 ("write") appears 1 time.
Word 49 ("click") appears 1 time.
Word 96 ("leav") appears 1 time.
Word 164 ("address") appears 1 time.
Word 230 ("regist") appears 1 time.
Word 260 ("internet") appears 1 time.
Word 296 ("messag") appears 1 time.
Word 320 ("support") appears 1 time.
Word 323 ("chang") appears 1 time.
Word 325 ("devic") appears 1 time.
Word 331 ("secur") appears 2 time.
Word 432 ("option") appears 2 time.


In [38]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.2423440465578673),
 (1, 0.28042534231127497),
 (2, 0.18022717849633083),
 (3, 0.29376971285680803),
 (4, 0.21120326362048672),
 (5, 0.4067649614393323),
 (6, 0.31663194624956603),
 (7, 0.2734721360286706),
 (8, 0.3862224200620612),
 (9, 0.3237077398516211),
 (10, 0.3237077398516211)]


In [39]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [40]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"bounce_backloan" + 0.025*"open" + 0.024*"hsbc_uk" + 0.023*"account" + 0.022*"time" + 0.019*"bank" + 0.019*"feeder" + 0.018*"loan" + 0.018*"hsbcukbusi" + 0.017*"bounc"
Topic: 1 
Words: 0.044*"account" + 0.039*"hsbc_uk" + 0.035*"thank" + 0.034*"bounce_backloan" + 0.031*"busi" + 0.024*"hsbcukbusi" + 0.022*"https" + 0.021*"help" + 0.018*"feeder" + 0.017*"need"
Topic: 2 
Words: 0.051*"thank" + 0.028*"hello" + 0.027*"send" + 0.026*"account" + 0.021*"https" + 0.019*"link" + 0.018*"bank" + 0.018*"bounce_backloan" + 0.017*"team" + 0.016*"check"
Topic: 3 
Words: 0.056*"bounce_backloan" + 0.039*"bank" + 0.038*"thank" + 0.031*"check" + 0.031*"account" + 0.024*"https" + 0.021*"credit" + 0.020*"hsbc_uk" + 0.018*"feeder" + 0.017*"hsbc_in"
Topic: 4 
Words: 0.040*"account" + 0.032*"busi" + 0.031*"loan" + 0.031*"person" + 0.030*"custom" + 0.029*"feeder" + 0.029*"bounc" + 0.028*"barclay" + 0.027*"worst" + 0.026*"absolut"
Topic: 5 
Words: 0.033*"bank" + 0.029*"https" + 0.025*"bounc

In [41]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.012*"bounce_backloan" + 0.011*"applic" + 0.011*"account" + 0.011*"https" + 0.011*"appli" + 0.010*"thank" + 0.009*"know" + 0.009*"hsbc_uk" + 0.009*"receiv" + 0.009*"team"
Topic: 1 Word: 0.018*"bank" + 0.017*"https" + 0.016*"account" + 0.015*"hsbc_uk" + 0.014*"bounce_backloan" + 0.011*"wait" + 0.010*"feeder" + 0.010*"tata_cruc" + 0.010*"busi" + 0.009*"thank"
Topic: 2 Word: 0.038*"https" + 0.014*"bank" + 0.013*"bounce_backloan" + 0.013*"hsbc_uk" + 0.012*"today" + 0.010*"lloydsbank" + 0.010*"sign" + 0.010*"week" + 0.008*"appli" + 0.008*"account"
Topic: 3 Word: 0.033*"bounce_backloan" + 0.016*"hsbc_uk" + 0.015*"thank" + 0.011*"https" + 0.010*"great" + 0.010*"appli" + 0.010*"email" + 0.009*"hello" + 0.009*"account" + 0.008*"look"
Topic: 4 Word: 0.018*"thank" + 0.016*"hsbc_uk" + 0.014*"send" + 0.014*"hello" + 0.010*"bank" + 0.010*"servic" + 0.009*"https" + 0.009*"detail" + 0.009*"account" + 0.009*"team"
Topic: 5 Word: 0.019*"thank" + 0.017*"hsbc_in" + 0.015*"work" + 0.013*"hs

In [42]:
processed_docs[4310]

['debasmita',
 'hello',
 'debasmita',
 'thank',
 'write',
 'address',
 'chang',
 'option',
 'regist',
 'internet',
 'bank',
 'secur',
 'devic',
 'click',
 'help',
 'support',
 'leav',
 'option',
 'servicesâ',
 'send',
 'secur',
 'messag']

In [43]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.7802041172981262	 
Topic: 0.055*"bounce_backloan" + 0.032*"hsbc_uk" + 0.030*"account" + 0.026*"feeder" + 0.022*"appli" + 0.021*"hsbcukbusi" + 0.020*"wait" + 0.019*"option" + 0.017*"applic" + 0.016*"thank"

Score: 0.1797870248556137	 
Topic: 0.051*"thank" + 0.028*"hello" + 0.027*"send" + 0.026*"account" + 0.021*"https" + 0.019*"link" + 0.018*"bank" + 0.018*"bounce_backloan" + 0.017*"team" + 0.016*"check"


In [44]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.4342068135738373	 
Topic: 0.018*"thank" + 0.016*"hsbc_uk" + 0.014*"send" + 0.014*"hello" + 0.010*"bank" + 0.010*"servic" + 0.009*"https" + 0.009*"detail" + 0.009*"account" + 0.009*"team"

Score: 0.3746648132801056	 
Topic: 0.033*"hsbc_uk" + 0.029*"thank" + 0.024*"bounce_backloan" + 0.022*"hsbcukbusi" + 0.021*"account" + 0.016*"https" + 0.015*"feeder" + 0.013*"appli" + 0.013*"email" + 0.010*"busi"

Score: 0.15611757338047028	 
Topic: 0.019*"thank" + 0.017*"hsbc_in" + 0.015*"work" + 0.013*"hsbc_uk" + 0.013*"help" + 0.012*"https" + 0.010*"branch" + 0.009*"charg" + 0.009*"time" + 0.009*"tell"


In [45]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.36760929226875305	 Topic: 0.056*"hsbc_uk" + 0.054*"bounce_backloan" + 0.054*"thank" + 0.049*"email" + 0.020*"wait"
Score: 0.3656103014945984	 Topic: 0.033*"bank" + 0.029*"https" + 0.025*"bounce_backloan" + 0.025*"email" + 0.022*"thank"
Score: 0.03335218131542206	 Topic: 0.036*"bounce_backloan" + 0.031*"https" + 0.026*"bank" + 0.025*"week" + 0.020*"thank"
Score: 0.03335188701748848	 Topic: 0.050*"bounce_backloan" + 0.024*"appli" + 0.023*"loan" + 0.019*"bounc" + 0.017*"custom"
Score: 0.03335050866007805	 Topic: 0.044*"account" + 0.039*"hsbc_uk" + 0.035*"thank" + 0.034*"bounce_backloan" + 0.031*"busi"
Score: 0.03334813937544823	 Topic: 0.040*"account" + 0.032*"busi" + 0.031*"loan" + 0.031*"person" + 0.030*"custom"
Score: 0.03334522247314453	 Topic: 0.055*"bounce_backloan" + 0.032*"hsbc_uk" + 0.030*"account" + 0.026*"feeder" + 0.022*"appli"
Score: 0.033344198018312454	 Topic: 0.051*"thank" + 0.028*"hello" + 0.027*"send" + 0.026*"account" + 0.021*"https"
Score: 0.033344149589538574